# Very Simple Ridge Model Experiment

**CRITICAL INSIGHT**: ALL model families follow the SAME CV-LB relationship.
- LB = 4.23×CV + 0.0533 (R²=0.981)
- Intercept (0.0533) > Target (0.0347) → Current approach CANNOT reach target
- This suggests the problem is STRUCTURAL, not model-dependent

**Hypothesis**: A simpler model might have a LOWER INTERCEPT because it can't overfit to the training distribution.

**Implementation**:
- Use only Spange + Arrhenius kinetics features (18 features)
- Ridge Regression with alpha = 10.0 (strong L2 regularization)
- Per-target models (3 separate Ridge regressors)
- TTA for mixtures
- NO neural networks, NO ensembles

**Why this might work**:
- Ridge regression is a linear model with L2 regularization
- It can't overfit as much as neural networks
- The CV-LB gap might be due to overfitting
- A simpler model might have a lower intercept

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import torch
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)

print('Imports complete')

In [ ]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

In [ ]:
# Load Spange descriptors
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
print(f'Spange: {SPANGE_DF.shape}')

In [ ]:
# Simple Spange Featurizer (18 features: 2 + 3 + 13)
class SpangeFeaturizer:
    """Simple featurizer using only Spange descriptors + Arrhenius kinetics.
    
    Total features: 18
    - 2: Residence Time, Temperature
    - 3: Arrhenius kinetics (inv_temp, log_time, interaction)
    - 13: Spange descriptors
    """
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1]  # 18 features

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

print(f'Spange feature dimension: {SpangeFeaturizer().feats_dim}')

In [ ]:
# Very Simple Ridge Model
class SimpleRidgeModel:
    """Very simple Ridge regression model.
    
    Uses only Spange + Arrhenius kinetics features (18 features).
    Ridge Regression with strong L2 regularization (alpha = 10.0).
    Per-target models (3 separate Ridge regressors).
    
    Hypothesis: A simpler model might have a LOWER INTERCEPT in the CV-LB relationship.
    """
    def __init__(self, data='single', alpha=10.0):
        self.data_type = data
        self.alpha = alpha
        self.models = []  # One per target
        self.scaler = StandardScaler()
        self.featurizer = SpangeFeaturizer(mixed=(data == 'full'))
    
    def train_model(self, X, Y):
        """Train Ridge models (one per target)."""
        X_feat = self.featurizer.featurize(X)
        X_scaled = self.scaler.fit_transform(X_feat)
        Y_np = Y.values
        
        # Train 3 separate Ridge models (one per target)
        self.models = []
        for i in range(3):
            model = Ridge(alpha=self.alpha)
            model.fit(X_scaled, Y_np[:, i])
            self.models.append(model)
    
    def predict(self, X):
        """Predict using trained Ridge models."""
        X_feat = self.featurizer.featurize(X)
        X_scaled = self.scaler.transform(X_feat)
        
        # Get predictions from each model
        preds = np.column_stack([model.predict(X_scaled) for model in self.models])
        
        # Clip to valid range
        preds = np.clip(preds, 0, 1)
        
        return torch.tensor(preds)
    
    def predict_with_tta(self, X):
        """Predict with test-time augmentation for mixed solvents."""
        if self.data_type == 'single':
            return self.predict(X)
        
        # For mixed solvents, average predictions from both orderings
        pred1 = self.predict(X)
        
        # Create flipped version
        X_flip = X.copy()
        X_flip["SOLVENT A NAME"] = X["SOLVENT B NAME"]
        X_flip["SOLVENT B NAME"] = X["SOLVENT A NAME"]
        X_flip["SolventB%"] = 1 - X["SolventB%"]
        
        pred2 = self.predict(X_flip)
        
        # Average
        return (pred1 + pred2) / 2

print('SimpleRidgeModel defined with alpha=10.0')

In [ ]:
# Test the model
X_single, Y_single = load_data("single_solvent")
print(f"Single solvent data: {X_single.shape[0]} samples, {len(X_single['SOLVENT NAME'].unique())} unique solvents")

X_full, Y_full = load_data("full")
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
print(f"Full data: {X_full.shape[0]} samples, {len(ramps)} unique ramps")

# Quick test
model = SimpleRidgeModel(data='single')
model.train_model(X_single.head(100), Y_single.head(100))
pred = model.predict(X_single.head(10))
print(f"Test prediction shape: {pred.shape}")

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = SimpleRidgeModel(data='single', alpha=10.0)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = SimpleRidgeModel(data='full', alpha=10.0)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict_with_tta(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
# Calculate CV score (for verification only - NOT part of submission)
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Get actuals in same order as predictions
actuals_single = []
for solvent in sorted(X_single["SOLVENT NAME"].unique()):
    mask = X_single["SOLVENT NAME"] == solvent
    actuals_single.append(Y_single[mask].values)
actuals_single = np.vstack(actuals_single)

actuals_full = []
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
for _, row in ramps.iterrows():
    mask = (X_full["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X_full["SOLVENT B NAME"] == row["SOLVENT B NAME"])
    actuals_full.append(Y_full[mask].values)
actuals_full = np.vstack(actuals_full)

# Get predictions
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== SIMPLE RIDGE MODEL CV SCORE ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')

print(f'\n=== COMPARISON ===')
print(f'exp_032 (best CV, GP 0.15 + MLP 0.55 + LGBM 0.3): CV 0.008194')
print(f'exp_049 (SIMPLE RIDGE): CV {overall_mse:.6f}')

if overall_mse < 0.008194:
    improvement = (0.008194 - overall_mse) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than exp_032!')
else:
    degradation = (overall_mse - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than exp_032')
    
# Estimate LB using old relationship
estimated_lb = 4.23 * overall_mse + 0.0533
print(f'\nEstimated LB (using old relationship): {estimated_lb:.4f}')
print(f'Best LB so far: 0.0877')
print(f'Target: 0.0347')
print(f'\nKey question: Does Simple Ridge have a DIFFERENT CV-LB relationship?')
print(f'If it has lower intercept, we might be able to reach the target.')